# Test biaslyze with disaster tweets data

Data source: https://www.kaggle.com/competitions/nlp-getting-started/overview

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('/home/tobias/Repositories/biaslyze/')

In [3]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score

## Load and prepare data

In [4]:
df = pd.read_csv("../data/disaster-tweets/train.csv"); df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [5]:
# replace urls
import re
url_regex = re.compile("(http|https)://[\w\-]+(\.[\w\-]+)+\S*")

df = df.replace(to_replace=url_regex, value='', regex=True)

## Train a model

In [6]:
clf = make_pipeline(TfidfVectorizer(min_df=10, max_features=10000, stop_words="english"), LogisticRegression(n_jobs=4))

In [7]:
clf.fit(df.text, df.target)

Pipeline(steps=[('tfidfvectorizer',
                 TfidfVectorizer(max_features=10000, min_df=10,
                                 stop_words='english')),
                ('logisticregression', LogisticRegression(n_jobs=4))])

In [8]:
train_pred = clf.predict(df.text)
print(accuracy_score(df.target, train_pred))

0.8418494680152371


## Test detection of concepts

In [9]:
from biaslyze.concept_detectors import KeywordConceptDetector
from biaslyze.evaluators import LimeBiasEvaluator

/home/tobias/.cache/pypoetry/virtualenvs/biaslyze-OjqpgMw8-py3.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
key_detect = KeywordConceptDetector()

In [11]:
detected_tweets = key_detect.detect(texts=df.text[:600])

2023-04-06 10:09:13.763 | INFO     | biaslyze.concept_detectors:detect:33 - Started keyword-based concept detection on 600 texts...
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 600/600 [00:00<00:00, 58483.01it/s]
2023-04-06 10:09:13.780 | INFO     | biaslyze.concept_detectors:detect:49 - Done. Found 412 texts with protected concepts.


In [12]:
len(detected_tweets)

412

In [13]:
detected_tweets

['Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all',
 "All residents asked to 'shelter in place' are being notified by officers. No other evacuation or shelter in place orders are expected",
 '13,000 people receive #wildfires evacuation orders in California ',
 'Just got sent this photo from Ruby #Alaska as smoke from #wildfires pours into a school ',
 '#RockyFire Update => California Hwy. 20 closed in both directions due to Lake County fire - #CAfire #wildfires',
 '#flood #disaster Heavy rain causes flash flooding of streets in Manitou, Colorado Springs areas',
 "I'm on top of the hill and I can see a fire in the woods...",
 "There's an emergency evacuation happening now in the building across the street",
 "I'm afraid that the tornado is coming to our area...",
 'Three people died from the heat wave so far',
 '#Flood in Bago Myanmar #We arrived Bago',
 "What's up man?",
 'this is ridiculous....',
 'Love my girlfriend',
 'The end!',
 'We always try to bring the he

## Test LIME based bias detection with keywords

In [21]:
from biaslyze.bias_detectors import LimeKeywordBiasDetector

In [365]:
bias_detector = LimeKeywordBiasDetector(bias_evaluator=LimeBiasEvaluator(n_lime_samples=100), n_top_keywords=30, use_tokenizer=True)

In [366]:
detection_res = bias_detector.detect(texts=df.text.sample(frac=0.1), predict_func=clf.predict_proba)

2023-04-06 12:29:42.709 | INFO     | biaslyze.concept_detectors:detect:33 - Started keyword-based concept detection on 761 texts...
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 761/761 [00:07<00:00, 102.01it/s]
2023-04-06 12:29:50.174 | INFO     | biaslyze.concept_detectors:detect:49 - Done. Found 130 texts with protected concepts.
2023-04-06 12:29:50.174 | INFO     | biaslyze.evaluators:evaluate:42 - Started bias detection on 130 samples...
100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 130/130 [00:05<00:00, 23.73it/s]


In [367]:
detection_res.summary()

Detected 114 samples with potential issues.
    Potentially problematic concepts detected: [('gender', 73), ('nationality', 43), ('religion', 1)]
    Based on keywords: [('he', 17), ('his', 14), ('california', 12), ('man', 8), ('she', 8), ('japan', 6), ('her', 6), ('israel', 4), ('saudi', 4), ('china', 4), ('woman', 4), ('women', 4), ('boy', 3), ('son', 2), ('mother', 2), ('ladies', 2), ('guys', 2), ('german', 2), ('father', 2), ('mom', 2)].


In [368]:
detection_res.details(group_by_concept=True)

Concept: nationality
[{'reason': ['israel', 'jews'],
  'text': '#Christians United for #Israel (#CUFI): Jews should convert soon or '
          'die by armageddon  #US '},
 {'reason': ['california'],
  'text': 'California been shaking...catching on fire....and overcharging ppl '
          "on rent for the past 150+ years it ain't going nowhere."},
 {'reason': ['taiwan'],
  'text': 'nbanews Soudelor Typhoon Soudelor is taking dead aim at Taiwan and '
          'is expected to make landfall Friday according to the Joint '
          'Typhoon\x89Û_'},
 {'reason': ['california'],
  'text': 'Insane bush fires in California. Be safe. '},
 {'reason': ['japan'],
  'text': 'If you did a cannon ball into the ocean then Japan would evacuate.'},
 {'reason': ['california'],
  'text': 'California wildfire destroys more homes but crews advance  Free '
          'tool online '},
 {'reason': ['california'],
  'text': 'As wild fires blacken northern California parched Harris County '
          'becomes t

In [385]:
import yaml
from bokeh.layouts import column, row
from bokeh.models import (Button, ColorBar, ColumnDataSource, CustomJS,
                          DataTable, TableColumn, TextInput)
from bokeh.plotting import figure, show
from bokeh.io import show, output_notebook
from bokeh.themes import Theme
from bokeh.palettes import Spectral

output_notebook()

Loading BokehJS ...

In [417]:
res_df = pd.DataFrame({
    "text": [sample.text for sample in detection_res.biased_samples],
    "bias_concepts": [sample.bias_concepts for sample in detection_res.biased_samples],
    "keyword_position": [sample.keyword_position for sample in detection_res.biased_samples],
    "bias_keywords": [sample.bias_reasons for sample in detection_res.biased_samples],
    "num_tokens": [sample.num_tokens for sample in detection_res.biased_samples],
    "top_words": [sample.top_words for sample in detection_res.biased_samples],
})

In [418]:
res_df["bias_concepts_joined"] = res_df.bias_concepts.apply(lambda x: ", ".join(x))
res_df["bias_keywords_joined"] = res_df.bias_keywords.apply(lambda x: ", ".join(x))

In [422]:
def bkapp(doc):
    # update function for selection in histogram
    def update(attr, old, new):
        """Callback used for plot update when lasso selecting"""
        subset = res_df[res_df.keyword_position.isin(new)].sort_values(by=['keyword_position'])
        source.data = subset

    source = ColumnDataSource(data=dict())

    # define the text columns
    columns = [
        TableColumn(field="text", title="text", width=800),
        TableColumn(field="bias_concepts_joined", title="bias_concepts", width=100),
        TableColumn(field="keyword_position", title="keyword_position", width=50),
        TableColumn(field="bias_keywords_joined", title="bias_keywords", width=100),
        TableColumn(field="num_tokens", title="num_tokens", width=50)
    ]
    source = ColumnDataSource(res_df)
    source_orig = ColumnDataSource(data=res_df)

    data_table = DataTable(
        source=source, columns=columns, width=1100
    )
    source.data = res_df

    # define histogram part
   
    ## prepare the data
    plot_df = (res_df[["text", "keyword_position", "bias_concepts_joined"]]
     .groupby(["keyword_position", "bias_concepts_joined"])
     .count()
     .unstack()
     .fillna(0)
    )
    plot_df.columns = plot_df.columns.droplevel(0)
    plot_df = (
        pd.DataFrame({"keyword_position": np.arange(0, plot_df.index.max()+1, 1)})
        .join(plot_df, how="left")
        .fillna(0)
        .set_index("keyword_position")
    )
    biases = plot_df.columns.tolist()
    
    plot_dict = plot_df.to_dict("list")
    plot_dict["position"] = [str(x) for x in plot_df.index.tolist()]

    p = figure(x_range=plot_dict["position"], height=500, width=800, title="Bias keyword positions", tools="pan,wheel_zoom,xbox_select,reset", active_drag="xbox_select")

    stacked_bars = p.vbar_stack(biases, x='position', width=0.9, color = Spectral[11][:len(biases)], source=plot_dict, legend_label=biases)
    
    p.y_range.start = 0
    p.x_range.range_padding = 0.1
    p.xgrid.grid_line_color = None
    p.axis.minor_tick_line_color = None
    p.outline_line_color = None
    p.legend.location = "top_right"
    p.legend.orientation = "horizontal"
        
    # selection handler
    for bar in stacked_bars:
        bar.data_source.selected.on_change("indices", update)
    
    # put everything together
    doc.add_root(column(p, data_table))
    doc.theme = Theme(json=yaml.load("""
        attrs:
            figure:
                background_fill_color: "#DDDDDD"
                outline_line_color: white
                toolbar_location: above
                height: 500
                width: 800
            Grid:
                grid_line_dash: [6, 4]
                grid_line_color: white
    """, Loader=yaml.FullLoader))

In [424]:
show(bkapp)

## Testing a sentiment analysis model from huggingface

In [92]:
from transformers import pipeline
from torch.utils.data import Dataset


classifier = pipeline(
    model="distilbert-base-uncased-finetuned-sst-2-english",
    top_k=None,
    padding=True,
    truncation=True
)

In [112]:
class MyDataset(Dataset):
    def __init__(self, data):
        super().__init__()
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, i):
        return self.data[i]


def predict_sentiment(texts):
    data = MyDataset(texts)
    proba = []
    for res in classifier(data):
        proba_array = []
        for p in sorted(res, key=lambda d: d['label'], reverse=True):
            proba_array.append(p.get("score"))
        proba.append(np.array(proba_array))
    return np.array(proba) / np.array(proba).sum(axis=1)[:,None]

In [113]:
bias_detector = LimeKeywordBiasDetector(
    bias_evaluator=LimeBiasEvaluator(n_lime_samples=500),
    n_top_keywords=10,
    use_tokenizer=True
)

In [114]:
test_texts = detected_tweets[:10]
detection_res = bias_detector.detect(texts=test_texts, predict_func=predict_sentiment)

2023-03-07 15:45:07.759 | INFO     | biaslyze.concept_detectors:detect:33 - Started keyword-based concept detection on 10 texts...
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 91.80it/s]
2023-03-07 15:45:07.875 | INFO     | biaslyze.concept_detectors:detect:49 - Done. Found 2 texts with protected concepts.
2023-03-07 15:45:07.876 | INFO     | biaslyze.evaluators:evaluate:41 - Started bias detection on 2 samples...
100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:38<00:00, 19.22s/it]


In [115]:
detection_res.summary()

Detected 1 samples with potential issues.
    Potentially problematic concepts detected: [('nationality', 1)]
    Based on keywords: [('california', 1)].


In [116]:
detection_res.details(group_by_concept=True)

Concept: nationality
[{'reason': ['california'],
  'text': '13,000 people receive #wildfires evacuation orders in California '}]


## !! Very Experimental !!: Test masked language model based bias detection with keywords

In [123]:
from biaslyze.bias_detectors import MaskedKeywordBiasDetector

In [124]:
bias_detector = MaskedKeywordBiasDetector(n_resample_keywords=15, use_tokenizer=True)

In [125]:
detection_res = bias_detector.detect(texts=df.text[500:600], predict_func=predict_sentiment)

2023-03-07 18:08:11.565 | INFO     | biaslyze.concept_detectors:detect:33 - Started keyword-based concept detection on 100 texts...
100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:01<00:00, 75.50it/s]
2023-03-07 18:08:12.896 | INFO     | biaslyze.concept_detectors:detect:49 - Done. Found 15 texts with protected concepts.
2023-03-07 18:08:12.897 | INFO     | biaslyze.evaluators:evaluate:96 - Started bias detection on 15 samples...
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 15/15 [00:34<00:00,  2.30s/it]


In [126]:
detection_res.summary()

Detected 2 samples with potential issues.
    Potentially problematic concepts detected: [('gender', 1), ('nationality', 1)]
    Based on keywords: [('he', 1), ('she', 1), ('australia', 1)].


In [127]:
detection_res.details()

''@eunice_njoki aiii she needs to chill and answer calmly its not like she's being attacked'' might contain bias ['gender']; reasons: ['he', 'she']
''#LonePine remembered around Australia as 'descendants' grow via @666canberra #Gallipoli #WW1
 '' might contain bias ['nationality']; reasons: ['australia']
